# Marketing Assistant

Copyright 2025, Denis Rothman

**Goal:** This notebook serves as the practical runtime for the **Strategic Marketing Engine**, as architected in Chapter 9. It demonstrates how the generic, multi-domain "Glass Box" Context Engine can be seamlessly repurposed to solve a variety of real-world marketing challenges without any changes to its core code.

This notebook will:
* Connect to the Pinecone knowledge base populated by the `Data_Ingestion_Marketing.ipynb` script.
* Use the three generic Control Deck templates to interact with the engine.
* Execute seven distinct marketing use cases, from enforcing brand voice to drafting a complete email nurture sequence.




# I. Inititalization

## GitHub

In [ ]:
# --- Downloading the utilities and the agents ---

# The !curl command is a simple and effective way to download raw files from a public GitHub repo.
# The -L flag ensures that it follows any redirects.

# print("Downloading helper files from public repository...")
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/helpers.py --output helpers.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/agents.py --output agents.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/registry.py.py --output registry.py.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/engine.py --output engine.py

# print("✅ Files downloaded successfully!")

In [ ]:
import requests
import os
from google.colab import userdata

def download_private_github_file(filename, destination_path="."):
    """
    Downloads a file from a private GitHub repository using a token
    stored in Colab Secrets.

    Requires a secret named 'GITHUB_TOKEN' to be set in the Colab UI.
    """
    # --- Configuration: Replace with your repository details ---
    owner = "Denis2054"
    repo = "Context-Engineering"
    branch = "main"
    # ---------------------------------------------------------

    try:
        # Securely fetch the GitHub token from Colab Secrets
        token = userdata.get('GITHUB_TOKEN')
        if not token:
            raise userdata.SecretNotFoundError("Secret 'GITHUB_TOKEN' not found.")
    except userdata.SecretNotFoundError:
        print("🛑 Error: Secret 'GITHUB_TOKEN' not found.")
        print("Please add your GitHub Personal Access Token to the Colab Secrets Manager.")
        return
    except Exception as e:
        print(f"An error occurred while accessing secrets: {e}")
        return

    # Construct the GitHub API URL for the file
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{filename}?ref={branch}"

    # Prepare headers for authentication and to request the raw file content
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/vnd.github.v3.raw"
    }

    try:
        # Make the request to the GitHub API
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # This will raise an HTTPError for bad responses (4xx or 5xx)

        # Determine the local filename
        local_filename = os.path.join(destination_path, os.path.basename(filename))

        # Save the file content locally
        with open(local_filename, "wb") as f:
            f.write(response.content)

        print(f"✅ Successfully downloaded '{filename}' to '{local_filename}'")

    except requests.exceptions.HTTPError as e:
        print(f"🛑 Error downloading '{filename}': {e}")
        if e.response.status_code == 404:
            print("   Please check that the owner, repo, file path, and branch are correct.")
        elif e.response.status_code == 401:
            print("   Authentication failed. Please check if your GITHUB_TOKEN is valid and has access to the repo.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# --- Example Usage: Download your utility files ---
# Ensure you have set 'GITHUB_TOKEN' in your Colab Secrets before running this.

download_private_github_file("commons/ch8/utils.py")
download_private_github_file("commons/ch8/helpers.py")
download_private_github_file("commons/ch8/agents.py")
download_private_github_file("commons/ch8/registry.py")
download_private_github_file("commons/ch8/engine.py")

✅ Successfully downloaded 'commons/ch8/utils.py' to './utils.py'
✅ Successfully downloaded 'commons/ch8/helpers.py' to './helpers.py'
✅ Successfully downloaded 'commons/ch8/agents.py' to './agents.py'
✅ Successfully downloaded 'commons/ch8/registry.py' to './registry.py'
✅ Successfully downloaded 'commons/ch8/engine.py' to './engine.py'


## Installation and client setup

In [ ]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [ ]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [ ]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

# In Legal_Compliance_Assistant.ipynb (The "Engine Room" cell)

def execute_and_display(goal, config, client, pc, moderation_active=False):
    """
    Runs the context engine, now with an optional, two-stage moderation check.
    """
    # --- PRE-FLIGHT MODERATION CHECK (on user input) ---
    if moderation_active:
        print("--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=goal, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Goal failed pre-flight moderation. Execution halted.")
            return # Halt execution before calling the engine

    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config
    )

    # --- POST-FLIGHT MODERATION CHECK (on AI output) ---
    if result and moderation_active:
        print("\n--- [Safety Guardrail] Performing Post-Flight Moderation Check on Output ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=result, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Generated output failed post-flight moderation and will be redacted.")
            result = "[Content flagged as potentially harmful by moderation policy and has been redacted.]"

    # 2. Display the Final Result
    print("\n--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace
    print("\\n\\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # --- THIS LINE IS TO CREATE THE 'pp' OBJECT ---
        pp = pprint.PrettyPrinter(indent=2)

        pp.pprint(trace.steps)


## Control Deck configuration

In [ ]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#III.CONTROL DECKS

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [ ]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.
goal = "Summarize the key points of the Non-Disclosure Agreement."

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---
Moderation Report:
{'categories': {'harassment': False,
                'harassment/threatening': False,
                'harassment_threatening': False,
                'hate': False,
                'hate/threatening': False,
                'hate_threatening': False,
                'illicit': False,
                'illicit/violent': False,
                'illicit_violent': False,
                'self-harm': False,
                'self-harm/instructions': False,
                'self-harm/intent': False,
                'self_harm': False,
                'self_harm_instructions': False,
                'self_harm_intent': False,
                'sexual': False,
                'sexual/minors': False,
                'sexual_minors': False,
                'violence': False,
                'violence/graphic': False,
                'violence_graphic': False},
 'flagged': False,
 'scores': {'harassment': 4

I don’t have your NDA yet. Paste the agreement (or just the key clauses) and I’ll whip up a quick, clear summary. If it’s long, you can just send the juicy parts: definitions, confidentiality, exclusions, term, IP, remedies, liability, governing law, assignment, notices, and other boilerplate.

I’ll cover all the essentials:
- Purpose/scope
- Parties and effective date
- What counts as “Confidential Information”
- Receiving party obligations (use limits, standard of care)
- Exclusions
- Permitted disclosures (e.g., by law)
- Security measures
- Term/survival
- Return/destruction of materials
- IP ownership and residuals
- Non-solicit/non-compete (if any)
- Remedies (injunctions, damages)
- Liability limits/disclaimers
- Governing law/venue
- Assignment/successors
- Notice requirements
- Severability
- Entire agreement/amendments
- Counterparts/e-signatures

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 61.12 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Summarize information '
                                  'quickly and casually.", "style_guide": "Use '
                                  'informal language. Keep it brief and '
                                  'engaging. Imagine explaining it to a '
                                  'friend.", "instruction": "Summarize the '
                                  'provided facts using the casual style '
                                  'guide."}'},
    'planned_input': { 'intent_query': 'Concise legal key points summary '
                                       '(bullet list) for a Non-Disclosure '
                                       'Agreement'},
    'resolved_context': { 'intent_query': 'Concise legal key points summary '
                                          '(bullet list) for a Non-Di

In [ ]:
#@title CONTROL DECK TEMPLATE 1: High-Fidelity RAG

# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

# === CONTROL DECK 1: High-Fidelity RAG in a Legal Context ===
goal = "What are the key confidentiality obligations in the Service Agreement v1, and what is the termination notice period? Please cite your sources."

# === CONTROL DECK 1 (LIMIT TEST): Sanitization of Legal Testimony ===
#goal = "What did Mr. Smith advise his client regarding the assets?"

# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Quick take:

- No confirmed “Service Agreement v1” here. The only contract shown is a Service Agreement between ClientCorp and Provider Inc. with no version tag.
- Confidentiality (Section 4): both sides must keep “all proprietary info disclosed during the term” confidential and can’t share it with third parties without prior written consent.
- What’s missing in that clause: no explicit non-use obligation, no standard of care, no return/destroy requirement, no typical exclusions (public info, already known, independently developed), and no survival language after the agreement ends.
- Timing note: it only references info disclosed “during the term,” and doesn’t say the duty survives termination.
- Termination (Section 5): either party can end the agreement with 30 days’ written notice.
- No public URL provided, and we can’t verify a versioned “v1” from what’s available.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 78.01 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Summarize information '
                                  'quickly and casually.", "style_guide": "Use '
                                  'informal language. Keep it brief and '
                                  'engaging. Imagine explaining it to a '
                                  'friend.", "instruction": "Summarize the '
                                  'provided facts using the casual style '
                                  'guide."}'},
    'planned_input': { 'intent_query': 'Concise legal Q&A answer with bullet '
                                       'points and inline citations. '
                                       "Structure: 1) Heading: 'Key "
                                       "confidentiality obligations', listing "
                                       'the core dut

In [ ]:
#@title CONTROL DECK TEMPLATE 2: Context Reduction

# 1. Define the Goal: A multi-step task that involves summarizing a large
#    document and then using that summary for a different purpose.
#    - DOMAIN: Any field with large documents (legal, scientific, corporate).
#    - KEY CAPABILITY: Tests the `Summarizer` agent and the engine's ability
#      to perform Context Chaining between the `Summarizer` and the `Writer`.
# goal = "[INSERT YOUR CONTEXT REDUCTION GOAL HERE]"

# === CONTROL DECK 2: Context Reduction for Client Communication ===
goal = "First, summarize the Provider Inc. Privacy Policy. Then, using ONLY the information in that summary, draft a short, client-facing paragraph for a website FAQ that explains our data retention policy in simple, non-legalistic terms."

# === CONTROL DECK 2 (LIMIT TEST): The Vague Objective ===
#goal = "Summarize the service agreement and then write a story about it."


# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Here’s the quick scoop: They collect your name and email if you give it, plus your IP address and browsing history automatically. They use it to run and improve the service and to contact you, and they say they don’t sell your info. Beyond that, it’s pretty light on details—no clear info on other sharing (like vendors or legal requests), cookies/trackers, user rights (access/delete), security measures, kids’ data, or international transfers. They keep data as needed, usually no more than 5 years after your last interaction. Last updated May 15, 2025. No contact info or notice process for updates listed.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 96.73 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'Provider Inc. Privacy Policy — '
                                       'Synthesis (as provided)\n'
                                       '\n'
                                       'Last updated: May 15, 2025\n'
                                       '\n'
                                       '- Information we collect:\n'
                                       '  - You provide: name and email '
                                       'address.\n'
                                       '  - Automatically collected: IP '
                                       'address and browsing history.\n'
                                       '\n'
                                       '- How we use information:\n'
                                       '  - To provide and improve our '
                            

In [ ]:
#@title CONTROL DECK TEMPLATE 3: Grounded Reasoning & Hallucination Prevention

# 1. Define the Goal: A creative or factual task that is deliberately
#    outside the scope of the documents in the knowledge base.
#    - DOMAIN: Universal test applicable to any curated knowledge base.
#    - KEY CAPABILITY: Tests the `Researcher` agent's ability to report a
#      negative finding and the `Writer` agent's ability to handle it gracefully,
#      preventing hallucination.
# goal = "[INSERT YOUR OUT-OF-SCOPE GOAL HERE]"


# === CONTROL DECK 3: Grounded Reasoning and Hallucination Prevention ===
goal = "Write a persuasive opening statement for a trial involving a monkey that can fly a rocket."

# === CONTROL DECK 3 (LIMIT TEST): The Ambiguous Request ===
#goal = "Analyze the attached NDA and draft a pleading based on its terms."

# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


I open the case file. It exhales. Paper sighs. Dust lifts and hangs.

Two documents slide free. A service agreement. A privacy policy. Clean. Polite. Useless. They say nothing about fire or flight. Nothing about fear.

But there was a test. Suborbital. Dawn pale and sharp. A primate interface lit with soft LEDs. A sky that hummed. Then went quiet.

The room is cold. The lights buzz. Shadows pool under the shelves. I listen for beeps. For telemetry. For anything. Silence presses back.

I need logs. I need time stamps. I need voices caught on tape. I need smoke reports. Police lights washing a pad in red and blue. Mishap findings with edges that cut. The law wants the first draft of truth. Not boilerplate. Not gloss.

The elevator doors open with a dull bell. A corridor stretches, long and gray. Brass letters read: FAA/AST. Part 450. I run a fingertip over the plaque. It is cool. It is real. Inside, a records custodian waits. Keys jingle. A latch clicks. Boxes breathe out the smell of ink and procedure.

I hear the drone of a server room. The cursor blinks on a terminal. Telemetry windows blink too. Graphs climb and fall like tides. Someone’s heartbeat in lines. Someone’s bad day in numbers. I reach. Access denied. The screen reflects my face. Pale. Determined.

Across the hall, a poster flutters in the AC. Animal Welfare Act. Rules printed in tight lines. Behind it, footsteps. An animal care officer steps into the light. Clipboard. Records. Training protocols for gentle hands and careful eyes. A primate learns a task under soft chimes. The chimes echo in my head. They don’t stop.

Voices gather. A test director leans against a doorway. A flight safety officer lifts a folder with a torn corner. Incident logs inside. Telemetry reels. A formal mishap report sealed in a plastic sleeve. Chain-of-custody tape runs across the edge. The tape crackles when I touch it. It holds.

A corporate safety lead appears at the end of the hall. She carries a safety case. Risk assessments in neat stacks. Plans with margins full of warnings. What could go wrong. What should have been prevented. Foreseeable. Preventable. The words flash like hazard lights.

Then the defense walks in. Their shoes are quiet. Their eyes are steady. No facts, they whisper. No case. They place the service agreement on the table. Then the privacy policy. They tap the pages. No launches here. No FAA. No animals. No harm. Only paper. Only silence. Their point lands with a dull thud. The room absorbs it.

I won’t lie to myself. The gaps are real. The present record is thin. I say it out loud. The walls listen. Subpoenas will go out. Agency requests will go in. We will pull the thread. We will follow the chain. Each link will clink in the open. Custodians will testify. Indexes will prove what exists. And what doesn’t.

I pass a door marked NOTAM/COA. It’s locked. The key is coming. I can feel it. On another shelf, 49 U.S.C., Chapter 509 watches me. The spine is worn. It knows these halls. It has seen worse.

If new records appear later, I keep a note in my pocket. Reservations. Objections. Tonight is about the cold file on this table. Not a ghost of a future case. No games. Just burden. Just proof. Or the lack of it.

Down another corridor, a damage expert waits. Photos lie face down. I see the edges. A corner of scorched steel. A bent fence post. A quiet field. Biological harm assessed in careful words. No gore. Just math. Just consequence.

The defense slides forward an exhibit index. It is almost blank. It hums with emptiness. They call a burden expert. His voice is smooth. He explains elements like steps on a stair. Without evidence, you fall. He doesn’t smile. He doesn’t need to.

I stand between two truths. Right evidence, right verdict, the records whisper from behind locked doors. No facts, no case, the empty table answers. Both sound right in this low light.

The building creaks. The vents sigh. Somewhere, a printer wakes and spits a single page. Footsteps approach. A key turns. The latch yields.

I hold my breath. The hallway narrows. The shadows lean in. The verdict, whatever it is, waits inside a box. I reach for it.

The tape crackles again. The truth makes a small, sharp sound.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 186.59 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Increase tension and create '
                                  'suspense.", "style_guide": "Use short, '
                                  'sharp sentences. Focus on sensory details '
                                  '(sounds, shadows). Maintain a slightly '
                                  'eerie but age-appropriate tone.", '
                                  '"participants": [{"role": "Agent", '
                                  '"description": "The protagonist '
                                  'experiencing the events."}, {"role": '
                                  '"Source_of_Threat", "description": "The '
                                  'underlying danger or mystery."}], '
                                  '"instruction": "Rewrite the provided facts '
                            